In [1]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

1-2我们知道，在BERT的输入中，cls是一个，sep是大于等于一个。

'[CLS]'必须出现在样本段落的开头，一个段落可以有一句话也可以有多句话，每句话的结尾必须是'[SEP]'。

例如：['[CLS]', 'this', 'is', 'blue', '[SEP]', 'that', 'is', 'red', '[SEP]']

我们需要对输入BERT的数据进行处理,例如：

words = [self.CLS_TOKEN] + words + [self.SEP_TOKEN]

3.Tokenizer
调用tokenizer，使用tokenizer分割输入，将tokens转为ids。如下：

self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese') 

words = self.bert_tokenizer.tokenize(''.join(words)) 

feature = self.bert_tokenizer.convert_tokens_to_ids(sent + [self.PAD_TOKEN for _ in range(max_sent_len - len(sent))])

如果觉得BERT给的token不够用，或希望在BERT给的token中加入自己的token，加入以下代码即可：

例如，想要加入大写字母：

self.bert_tokenizer.add_tokens([chr(i) for i in range(ord("A"), ord("Z") + 1)]) 

args.len_token = len(self.bert_tokenizer)

此处使用了len_token记录self.bert_tokenizer新的token大小，因为要对模型进行更新。

4.模型使用
调用BertModel，因为改变了tokenizer所以对模型的token参数进行更新，然后就可以正常使用BERT-Model啦！

self.BertModel = BertModel.from_pretrained('bert-base-chinese') 
#加入了A-Z，重新resize一下大小

self.BertModel.resize_token_embeddings(self.args.len_token)  
outputs = self.BertModel(input_ids=ii, token_type_ids=tti, attention_mask=am)

本文使用了预训练模型bert-base-chinese做例子，其余的预训练模型具体可参考

https://link.zhihu.com/?target=https%3A//github.com/google-research/bert

6.
若想对Bert进行fine-tuning，如果存在out-of-memory的问题，可能会用到GPU并行：

self.BertModel = nn.DataParallel(self.BertModel, device_ids=args.bert_gpu_ids, output_device=torch.cuda.current_device())


7.下载预训练模型
https://link.zhihu.com/?target=https%3A//github.com/ymcui/Chinese-BERT-wwm

8.train.txt下载
数据集选择的THUCNews，整理出18w条数据，10类新闻文本的中文分类问题（10分类），每类新闻数据量相等，为1.8w条，数据集来自
https://github.com/649453932/Bert-Chinese-Text-Classification-Pytorch/tree/master/THUCNews/data

In [7]:
import torch
import time 
import torch.nn as nn
import torch.nn.functional as F 
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertConfig, BertAdam
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
from torch.utils.data import *

path = "data/"
bert_path = "chinese_roberta_wwm_ext_pytorch/"
tokenizer = BertTokenizer(vocab_file=bert_path + "vocab.txt")  # 初始化分词器

预处理数据集

In [11]:
input_ids = []     # input char ids
input_types = []   # segment ids
input_masks = []   # attention mask
label = []         # 标签
pad_size = 32      # 也称为 max_len (前期统计分析，文本长度最大值为38，取32即可覆盖99%)
 
with open(path + "train.txt", encoding='utf-8') as f:
    for i, l in tqdm(enumerate(f)): 
        try:
            x1, y = l.strip().split('\t')
        except:
            continue
        x1 = tokenizer.tokenize(x1)
        tokens = ["[CLS]"] + x1 + ["[SEP]"]
        
        # 得到input_id, seg_id, att_mask
        ids = tokenizer.convert_tokens_to_ids(tokens)
        types = [0] *(len(ids))
        masks = [1] * len(ids)
        # 短则补齐，长则切断
        if len(ids) < pad_size:
            types = types + [1] * (pad_size - len(ids))  # mask部分 segment置为1
            masks = masks + [0] * (pad_size - len(ids))
            ids = ids + [0] * (pad_size - len(ids))
        else:
            types = types[:pad_size]
            masks = masks[:pad_size]
            ids = ids[:pad_size]
        input_ids.append(ids)
        input_types.append(types)
        input_masks.append(masks)
#         print(len(ids), len(masks), len(types)) 
        assert len(ids) == len(masks) == len(types) == pad_size
        label.append([int(y)])


0it [00:00, ?it/s]
782it [00:00, 7814.36it/s]
1627it [00:00, 7992.13it/s]
2513it [00:00, 8232.29it/s]
3381it [00:00, 8361.50it/s]
4248it [00:00, 8450.83it/s]
5133it [00:00, 8565.06it/s]
5978it [00:00, 8528.25it/s]
6814it [00:00, 8468.77it/s]
7629it [00:00, 8367.18it/s]
8436it [00:01, 6955.41it/s]
9252it [00:01, 7276.65it/s]
10071it [00:01, 7526.08it/s]
10946it [00:01, 7854.94it/s]
11775it [00:01, 7977.18it/s]
12583it [00:01, 7958.01it/s]
13386it [00:01, 7975.48it/s]
14189it [00:01, 7980.42it/s]
15019it [00:01, 8072.46it/s]
15867it [00:01, 8189.21it/s]
16689it [00:02, 8042.92it/s]
17514it [00:02, 8103.78it/s]
18346it [00:02, 8166.68it/s]
19179it [00:02, 8213.51it/s]
20002it [00:02, 8201.43it/s]
20823it [00:02, 8139.32it/s]
21642it [00:02, 8153.06it/s]
22472it [00:02, 8196.26it/s]
23317it [00:02, 8270.67it/s]
24160it [00:02, 8317.22it/s]
24993it [00:03, 8178.05it/s]
25812it [00:03, 8147.08it/s]
26628it [00:03, 7980.91it/s]
27457it [00:03, 8067.49it/s]
28283it [00:03, 8122.33it/s]
29097i

切分训练集和测试集

In [19]:
# 随机打乱索引
random_order = list(range(len(input_ids)))
np.random.seed(2020)   # 固定种子
np.random.shuffle(random_order)
print(random_order[:10])

# 4:1 划分训练集和测试集
input_ids_train = np.array([input_ids[i] for i in random_order[:int(len(input_ids)*0.8)]])
input_types_train = np.array([input_types[i] for i in random_order[:int(len(input_ids)*0.8)]])
input_masks_train = np.array([input_masks[i] for i in random_order[:int(len(input_ids)*0.8)]])
y_train = np.array([label[i] for i in random_order[:int(len(input_ids) * 0.8)]])
print(input_ids_train.shape, input_types_train.shape, input_masks_train.shape, y_train.shape)

input_ids_test = np.array([input_ids[i] for i in random_order[int(len(input_ids)*0.8):]])
input_types_test = np.array([input_types[i] for i in random_order[int(len(input_ids)*0.8):]])
input_masks_test = np.array([input_masks[i] for i in random_order[int(len(input_ids)*0.8):]])
y_test = np.array([label[i] for i in random_order[int(len(input_ids) * 0.8):]])
print(input_ids_test.shape, input_types_test.shape, input_masks_test.shape, y_test.shape)

[105597, 73370, 23494, 7028, 106320, 72718, 13070, 103120, 48846, 98990]
(102332, 32) (102332, 32) (102332, 32) (102332, 1)
(25584, 32) (25584, 32) (25584, 32) (25584, 1)


加载到高效的DataLoader

In [20]:
BATCH_SIZE = 16
train_data = TensorDataset(torch.LongTensor(input_ids_train), 
                           torch.LongTensor(input_types_train), 
                           torch.LongTensor(input_masks_train), 
                           torch.LongTensor(y_train))
train_sampler = RandomSampler(train_data)  
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(torch.LongTensor(input_ids_test), 
                          torch.LongTensor(input_types_test), 
                         torch.LongTensor(input_masks_test),
                          torch.LongTensor(y_test))
test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

定义bert模型

In [21]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(bert_path)  # /bert_pretrain/
        for param in self.bert.parameters():
            param.requires_grad = True  # 每个参数都要 求梯度
        self.fc = nn.Linear(768, 10)   # 768 -> 2

    def forward(self, x):
        context = x[0]  # 输入的句子   (ids, seq_len, mask)
        types = x[1]
        mask = x[2]  # 对padding部分进行mask，和句子相同size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        _, pooled = self.bert(context, token_type_ids=types, 
                              attention_mask=mask, 
                              output_all_encoded_layers=False) # 控制是否输出所有encoder层的结果
        out = self.fc(pooled)   # 得到10分类
        return out

可以发现，bert模型的定义由于高效简易的封装库存在，使得定义模型较为容易，如果想要在bert之后加入cnn/rnn等层，可在这里定义。

实例化bert模型

In [22]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(DEVICE)
print(model) 

Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
 

定义优化器

In [27]:
param_optimizer = list(model.named_parameters())  # 模型参数名字列表
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
# optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
NUM_EPOCHS = 3
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=0.05,
                     t_total=len(train_loader) * NUM_EPOCHS
                    )

定义训练函数和测试函数

In [25]:
def train(model, device, train_loader, optimizer, epoch):   # 训练模型
    model.train()
    best_acc = 0.0 
    for batch_idx, (x1,x2,x3, y) in enumerate(train_loader):
        start_time = time.time()
        x1,x2,x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        y_pred = model([x1, x2, x3])  # 得到预测结果
        model.zero_grad()             # 梯度清零
        loss = F.cross_entropy(y_pred, y.squeeze())  # 得到loss
        loss.backward()
        optimizer.step()
        if(batch_idx + 1) % 100 == 0:    # 打印loss
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'.format(epoch, (batch_idx+1) * len(x1), 
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader), 
                                                                           loss.item()))  # 记得为loss.item()

def test(model, device, test_loader):    # 测试模型, 得到测试集评估结果
    model.eval()
    test_loss = 0.0 
    acc = 0 
    for batch_idx, (x1,x2,x3, y) in enumerate(test_loader):
        x1,x2,x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        with torch.no_grad():
            y_ = model([x1,x2,x3])
        test_loss += F.cross_entropy(y_, y.squeeze())
        pred = y_.max(-1, keepdim=True)[1]   # .max(): 2输出，分别为最大值和最大值的index
        acc += pred.eq(y.view_as(pred)).sum().item()    # 记得加item()
    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
          test_loss, acc, len(test_loader.dataset),
          100. * acc / len(test_loader.dataset)))
    return acc / len(test_loader.dataset)

开始训练和测试

In [ ]:
best_acc = 0.0 
PATH = 'roberta_model.pth'  # 定义模型保存路径
for epoch in range(1, NUM_EPOCHS+1):  # 3个epoch
    train(model, DEVICE, train_loader, optimizer, epoch)
    acc = test(model, DEVICE, test_loader)
    if best_acc < acc: 
        best_acc = acc 
        torch.save(model.state_dict(), PATH)  # 保存最优模型
    print("acc is: {:.4f}, best acc is {:.4f}\n".format(acc, best_acc))

加载最优模型进行测试

In [ ]:
model.load_state_dict(torch.load("roberta_model.pth"))
acc = test(model, DEVICE, test_loader)

# 如果打比赛的话，下面代码也可参考
"""
# 测试集提交
PATH = "roberta_model.pth"
model.load_state_dict(torch.load(PATH))
def test_for_submit(model, device, test_loader):    # 测试模型
    model.eval()
    preds = []
    for batch_idx, (x1,x2,x3) in tqdm(enumerate(test_loader)):
        x1,x2,x3 = x1.to(device), x2.to(device), x3.to(device)
        with torch.no_grad():
            y_ = model([x1,x2,x3])
        pred = y_.max(-1, keepdim=True)[1].squeeze().cpu().tolist()   
        # .max() 2输出，分别为最大值和最大值的index
        preds.extend(pred) 
    return preds 
preds = test_for_submit(model, DEVICE, test_loader)
"""